In [2]:
!pip install google-cloud-aiplatform --upgrade --user --quiet
!pip install langchain==0.0.229 chromadb==0.3.26 pydantic==1.10.8 typing-inspect==0.8.0 typing_extensions==4.5.0 pandas datasets google-api-python-client pypdf faiss-cpu transformers config --upgrade --user --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.5/964.5

In [3]:
# Below libraries are required to build a SQL engine for BigQuery
!pip install --user SQLAlchemy==1.4.48 --quiet
!pip install --user sqlalchemy-bigquery --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.17.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.1.4 which is incompatible.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.48 which is incompatible.


In [4]:
# Install pandas
! pip install --user --quiet pandas

# Install HuggingFace Datasets
! pip install --user --quiet datasets

# Install Python client for Google Search API
! pip install --user --quiet google-api-python-client

# PDF loader
! pip install --user --quiet pypdf

# Install similarity search library
# TO-DO: Replace with vertex AI Matching Engine
! pip install --user --quiet faiss-cpu

In [5]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [1]:
PROJECT_ID = "iron-decorator-297513" # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
from google.cloud import aiplatform
from langchain.llms import VertexAI
import vertexai
aiplatform.init(project=PROJECT_ID, location=LOCATION)
vertexai.init(project=PROJECT_ID, location=LOCATION)


In [4]:
REQUESTS_PER_MINUTE = 100

llm = VertexAI(
    model_name='text-bison@001',
    max_output_tokens=1024,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)
#chat = VertexChat()

#mchat = VertexMultiTurnChat(max_output_tokens=1024)

#embedding = VertexEmbeddings(requests_per_minute=REQUESTS_PER_MINUTE)


In [10]:
#iron-decorator-297513.tg_Dataste.LLMsales
project_id = "iron-decorator-297513" # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
dataset_id = 'tg_Dataste' # @param {type:"string"}
table_name = 'LLMsales' # @param {type:"string"}
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import pandas as pd

In [11]:
table_uri = f"bigquery://{project_id}/{dataset_id}"
engine = create_engine(f"bigquery://{project_id}/{dataset_id}")

In [12]:
query=f"""SELECT * FROM `{project_id}.{dataset_id}.{table_name}`"""
engine.execute(query).first()

<ipython-input-12-c2f5a6a300c8>:2: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  engine.execute(query).first()


('INV-02363800013', datetime.date(2016, 12, 23), '3654', 'THE BEVERAGE SHOP / BELMOND', '207 E MAIN ST', 'BELMOND', '50421', '99', 'WRIGHT', '1062500.0', 'FLAVORED RUM', '370.0', 'PERNOD RICARD USA', '42717', 'MALIBU COCONUT RUM', 12, 1000, 10.49, 15.74, 4, 62.96, 4.0, 1.06)

In [13]:
from langchain import SQLDatabase, SQLDatabaseChain
from langchain.prompts.prompt import PromptTemplate

In [ ]:
## reference do not execute this
def bq_qna(question):
  #create SQLDatabase instance from BQ engine
  db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=[table_name])
  # db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=['bigquery-public-data.iowa_liquor_sales.sales'])

  #create SQL DB Chain with the initialized LLM and above SQLDB instance
  db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True)



# Set up langchain
db = SQLDatabase.from_uri(sqlalchemy_url)
llm = OpenAI(temperature=0, model="text-davinci-003")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
llm=llm,
toolkit=toolkit,
verbose=True,
top_k=1000,
)

In [14]:
def bq_qna(question):
  #create SQLDatabase instance from BQ engine
  db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=[table_name])
  # db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=['bigquery-public-data.iowa_liquor_sales.sales'])

  #create SQL DB Chain with the initialized LLM and above SQLDB instance
  db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True)


  _googlesql_prompt = """You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Final answer here"
  Only use the following tables:
  {table_info}

  If someone asks for specific month, use ActivityDate between current month's start date and current month's end date

  Question: {input}"""

  BigQuerySQL_PROMPT = PromptTemplate(
      input_variables=["input", "table_info", "top_k"],
      template=_googlesql_prompt,
  )

  #passing question to the prompt template
  final_prompt = BigQuerySQL_PROMPT.format(input=question, table_info=table_name, top_k=10000)

  #pass final prompt to SQL Chain
  output = db_chain(final_prompt)


  return output['result'], output['intermediate_steps'][0]

In [15]:
bq_qna('Count total number of invoices')



> Entering new  chain...
You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Final answer h

('27934721',
 {'input': 'You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.\n  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.\n  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.\n  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n  Use the following format:\n  Question: "Question here"\n  SQLQuery: "SQL Query to run"\n  SQLResult: "Result of the SQLQuery"\n  Answer: "Final an

In [16]:
bq_qna('give me the first 10 records of the table')



> Entering new  chain...
You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Final answer h

('The first 10 records of the table are:\n\ninvoice_and_item_number\tdate\tstore_number\tstore_name\taddress\tcity\tzip_code\tcounty_number\tcounty\tcategory\tcategory_name\tvendor_number\tvendor_name\titem_number\titem_description\tpack\tbottle_volume_ml\tstate_bottle_cost\tstate_bottle_retail\tbottles_sold\tsale_dollars\tvolume_sold_liters\tvolume_sold_gallons\nINV-39002000009\t2021-08-06\t5723\tCASEY\'S GENERAL STORE #2538-WAUKEE\t20 WEST HICKMAN ROAD\tWAUKEE\t50263.0\t25\tDALLAS\t1082100.0\tIMPORTED CORDIALS & LIQUEURS\t192\tMAST-JAGERMEISTER US, INC\t65259\tJAGERMEISTER LIQUEUR MINI MEISTERS\t12\t20\t4.93\t7.4\t1\t7.4\t0.02\t0.0\nINV-24519900032\t2020-01-13\t5726\tEHLINGER\'S VINTON EXPRESS\t310 K AVENUE SOUTH\tVINTON\t52349.0\t6\tBENTON\t1082000.0\tIMPORTED CORDIALS & LIQUEURS\t192\tMAST-JAGERMEISTER US, INC\t65259\tJAGERMEISTER LIQUEUR MINI MEISTERS\t12\t20\t4.93\t7.4\t1\t7.4\t0.02\t0.0\nINV-13922700032\t2018-08-20\t3778\tFAMILY PANTRY\t4538 LOWER BEAVER RD\tDES MOINES\t50310\t7

In [21]:
bq_qna('give me the list of county for the city  "WAUKEE"')[0]



> Entering new  chain...
You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Final answer h

'DALLAS'

In [23]:
bq_qna('rank the top 10 Category name for each county based on the Sale_dollars')



> Entering new  chain...
You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Final answer h

('Answer:',
 {'input': 'You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.\n  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.\n  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.\n  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n  Use the following format:\n  Question: "Question here"\n  SQLQuery: "SQL Query to run"\n  SQLResult: "Result of the SQLQuery"\n  Answer: "Final ans

#LETS Use the Vehicle tables that we used for our POC


In [ ]:
project_id = "iron-decorator-297513" # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
dataset_id = 'tg_Dataste' # @param {type:"string"}
table_name = ['Vehicle_ID_Table','Mileage','topSpeed'] # @param {type:"string"}

In [39]:
llm = VertexAI(
    model_name='text-bison@001',
    max_output_tokens=1024,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)



table_uri = f"bigquery://{project_id}/{dataset_id}"
engine = create_engine(f"bigquery://{project_id}/{dataset_id}")


def bq_qna(question):
  #create SQLDatabase instance from BQ engine
  db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=table_name)
  # db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=['bigquery-public-data.iowa_liquor_sales.sales'])

  #create SQL DB Chain with the initialized LLM and above SQLDB instance
  db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True)


  _googlesql_prompt = """You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table. VEHICLE_ID is only available in Vehicle_ID_Table

I have a collection of tables containing vehicle data, and I need your help to extract specific information. Please carefully consider the following details:
  Relevant Tables :
  •	Vehicle_ID_Table: Contains vehicle ID (a unique numerical identifier), vehicle number (alphanumeric code), smoke emission (numerical value in grams/km), and temperature (numerical value in degrees Celsius).
  •	topSpeed: Contains vehicle number, capacitor (numerical value in Farads), cc (numerical value representing engine displacement), and top speed (numerical value in km/h).
  •	Mileage: Contains vehicle number, make (vehicle manufacturer name), type (vehicle model name), and mileage (numerical value in km/L).
  Key Point:
  •	Vehicle ID is only available in Vehicle_ID_Table. To retrieve it for a specific vehicle, use the vehicle number to look it up in this Table.
  Examples of SQL Queries:
  •	Find the vehicle ID with the highest top speed.
  •	What is the mileage of the vehicle with the lowest smoke emission?
  •	List the make and model of vehicles with a capacitor value of 50 Farads.
  •	Compare the average temperature of vehicles with a cc value above 2000 and those below 2000.
  Context:
  •	These Tables belong to a vehicle maintenance database.
  •	Smoke emission and temperature data are used for environmental compliance tracking.
  •	Capacitor values are relevant for electrical systems.
  •	Top speed and mileage are performance-related metrics.
  Please be ready to answer various questions about the vehicles based on this data, while accurately handling Vehicle ID retrieval and understanding
  the relationships between different columns and files.

  Use the following format:
  Question: "Question here"
  SQL : "Create the SQL query and check if the necessary column is available before executing it"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Final answer here"
  Only use the following tables:
  {table_info}

  If someone asks for specific month, use ActivityDate between current month's start date and current month's end date

  Question: {input}"""

  BigQuerySQL_PROMPT = PromptTemplate(
      input_variables=["input", "table_info", "top_k"],
      template=_googlesql_prompt,
  )

  #passing question to the prompt template
  final_prompt = BigQuerySQL_PROMPT.format(input=question, table_info=table_name, top_k=10000)

  #pass final prompt to SQL Chain
  output = db_chain(final_prompt)


  return output['result'], output['intermediate_steps'][0]

In [32]:
bq_qna("vehcile id with maximum temperature")



> Entering new  chain...
You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

I have a collection of tables containing vehicle data, and I need your help to extract specific information. Please carefully consider the following 

('16',
 {'input': 'You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.\n  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.\n  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.\n  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n\nI have a collection of tables containing vehicle data, and I need your help to extract specific information. Please carefully consider the following det

In [34]:
bq_qna("vehicle number with maximum topspeed. Also find the corresponding vehicle id ")



> Entering new  chain...
You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

I have a collection of tables containing vehicle data, and I need your help to extract specific information. Please carefully consider the following 

('WXYZ234, 16',
 {'input': 'You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.\n  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.\n  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.\n  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n\nI have a collection of tables containing vehicle data, and I need your help to extract specific information. Please carefully consider the foll

In [41]:
bq_qna("Vehicle number with the Maximum Mileage, Find the corresponding vehicle id from the Vehicle_ID_Table ")



> Entering new  chain...
You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table. VEHICLE_ID is only available in Vehicle_ID_Table

I have a collection of tables containing vehicle data, and I need your help to extract specific info

DatabaseError: ignored